In [ ]:
import os
import random
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torchvision.utils import save_image, make_grid

from tqdm import tqdm
import lpips
from torchmetrics.image.fid import FrechetInceptionDistance

# Reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create output directory
os.makedirs("training_progress", exist_ok=True)

# Dataset with multiple domains
class MultiDomainDataset(Dataset):
    def __init__(self, root, domains, transform=None):
        self.root = root
        self.domains = domains
        self.transform = transform
        self.domain_images = {}
        
        for domain in domains:
            domain_dir = os.path.join(root, domain)
            self.domain_images[domain] = os.listdir(domain_dir)
        
        # Find the domain with minimum images
        self.min_length = min([len(images) for images in self.domain_images.values()])

    def __len__(self):
        return self.min_length

    def __getitem__(self, idx):
        images = {}
        domains = {}
        
        for i, domain in enumerate(self.domains):
            img_path = os.path.join(self.root, domain, self.domain_images[domain][idx % len(self.domain_images[domain])])
            img = Image.open(img_path).convert("RGB")
            
            if self.transform:
                img = self.transform(img)
                
            images[domain] = img
            
            # One-hot encoding for domain
            domain_label = torch.zeros(len(self.domains))
            domain_label[i] = 1
            domains[domain] = domain_label
            
        return images, domains

# Image transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Paths and dataset setup
path_to_dataset = "/kaggle/input/selfie2anime"  # Adjust path as needed
domains = ["trainA", "trainB"]  # Domain folders
dataset = MultiDomainDataset(path_to_dataset, domains, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Get fixed batch for visualization
fixed_dataloader = DataLoader(dataset, batch_size=4, shuffle=False)
fixed_batch = next(iter(fixed_dataloader))

# AdaIN-based Residual Block for Generator
class AdaINResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, style_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 1, 1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1)
        self.norm1 = AdaptiveInstanceNorm(style_dim, in_channels)
        self.norm2 = AdaptiveInstanceNorm(style_dim, out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        # Skip connection
        self.skip = nn.Conv2d(in_channels, out_channels, 1, 1, 0) if in_channels != out_channels else nn.Identity()
        
    def forward(self, x, s):
        out = self.norm1(x, s)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.norm2(out, s)
        out = self.relu(out)
        out = self.conv2(out)
        out = out + self.skip(x)
        return out

# Adaptive Instance Normalization
class AdaptiveInstanceNorm(nn.Module):
    def __init__(self, style_dim, num_features):
        super().__init__()
        self.norm = nn.InstanceNorm2d(num_features, affine=False)
        self.fc = nn.Linear(style_dim, num_features*2)
        
    def forward(self, x, s):
        h = self.fc(s)
        h = h.view(h.size(0), h.size(1), 1, 1)
        gamma, beta = torch.chunk(h, chunks=2, dim=1)
        return (1 + gamma) * self.norm(x) + beta

# Style Encoder
class StyleEncoder(nn.Module):
    def __init__(self, img_size=256, style_dim=64, num_domains=2):
        super().__init__()
        layers = []
        channels = 64
        
        # Down-sampling layers
        layers.append(nn.Conv2d(3, channels, 3, 1, 1))
        for _ in range(4):
            layers.append(nn.Conv2d(channels, channels*2, 4, 2, 1))
            layers.append(nn.LeakyReLU(0.2))
            channels *= 2
        
        self.shared = nn.Sequential(*layers)
        
        # Domain-specific style encoders
        self.domain_styles = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(channels, channels, 4, 2, 1),
                nn.LeakyReLU(0.2),
                nn.Conv2d(channels, channels, 3, 1, 1),
                nn.LeakyReLU(0.2),
                nn.AdaptiveAvgPool2d(1),
                nn.Flatten(),
                nn.Linear(channels, style_dim)
            ) for _ in range(num_domains)
        ])
        
    def forward(self, x, domain_idx):
        h = self.shared(x)
        return self.domain_styles[domain_idx](h)

# Mapping Network
class MappingNetwork(nn.Module):
    def __init__(self, latent_dim=16, style_dim=64, num_domains=2):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU()
        )
        
        self.domain_maps = nn.ModuleList([
            nn.Sequential(
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, style_dim)
            ) for _ in range(num_domains)
        ])
        
    def forward(self, z, domain_idx):
        h = self.shared(z)
        return self.domain_maps[domain_idx](h)

# Generator
class Generator(nn.Module):
    def __init__(self, img_size=256, style_dim=64, num_domains=2):
        super().__init__()
        self.style_dim = style_dim
        
        # Initial convolution
        self.from_rgb = nn.Conv2d(3, 64, 3, 1, 1)
        
        # Down-sampling layers
        self.down = nn.ModuleList()
        channels = 64
        for _ in range(2):
            self.down.append(nn.Sequential(
                nn.Conv2d(channels, channels*2, 4, 2, 1),
                nn.InstanceNorm2d(channels*2),
                nn.ReLU(inplace=True)
            ))
            channels *= 2
        
        # Bottleneck blocks
        self.bottleneck = nn.ModuleList()
        for _ in range(6):
            self.bottleneck.append(AdaINResBlock(channels, channels, style_dim))
        
        # Up-sampling layers
        self.up = nn.ModuleList()
        for _ in range(2):
            self.up.append(nn.Sequential(
                nn.ConvTranspose2d(channels, channels//2, 4, 2, 1),
                nn.InstanceNorm2d(channels//2),
                nn.ReLU(inplace=True)
            ))
            channels //= 2
        
        # Output layer
        self.to_rgb = nn.Sequential(
            nn.Conv2d(channels, 3, 3, 1, 1),
            nn.Tanh()
        )
        
    def forward(self, x, s):
        x = self.from_rgb(x)
        
        # Down-sampling
        for layer in self.down:
            x = layer(x)
        
        # Apply AdaIN ResBlocks with style
        for block in self.bottleneck:
            x = block(x, s)
        
        # Up-sampling
        for layer in self.up:
            x = layer(x)
        
        return self.to_rgb(x)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_size=256, num_domains=2):
        super().__init__()
        self.num_domains = num_domains
        
        # Shared layers
        self.shared = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 4, 2, 1),
            nn.LeakyReLU(0.2)
        )
        
        # Domain-specific output layers
        self.domains = nn.ModuleList([
            nn.Conv2d(512, 1, 3, 1, 1) for _ in range(num_domains)
        ])
        
    def forward(self, x, domain_idx):
        h = self.shared(x)
        return self.domains[domain_idx](h)

# Initialize models
latent_dim = 16
style_dim = 64
num_domains = len(domains)

generator = Generator(style_dim=style_dim, num_domains=num_domains).to(device)
mapping_network = MappingNetwork(latent_dim=latent_dim, style_dim=style_dim, num_domains=num_domains).to(device)
style_encoder = StyleEncoder(style_dim=style_dim, num_domains=num_domains).to(device)
discriminator = Discriminator(num_domains=num_domains).to(device)

# Losses and Optimizers
criterion_adv = nn.MSELoss()
criterion_recon = nn.L1Loss()
criterion_style = nn.L1Loss()

# Optimizers
optimizer_G = optim.Adam(
    list(generator.parameters()) + 
    list(mapping_network.parameters()) + 
    list(style_encoder.parameters()),
    lr=0.0001, betas=(0.0, 0.99)
)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.0, 0.99))

# Metrics
fid_metric = FrechetInceptionDistance(feature=64).to(device)
lpips_metric = lpips.LPIPS(net='vgg').to(device)

# Training loop
epochs = 100
lambda_sty = 1.0
lambda_ds = 1.0
lambda_cyc = 1.0

for epoch in range(epochs):
    generator.train()
    mapping_network.train()
    style_encoder.train()
    discriminator.train()
    
    for batch_idx, (images, domain_labels) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")):
        # Move data to device
        src_domain, trg_domain = domains[0], domains[1]
        x_real_src = images[src_domain].to(device)
        x_real_trg = images[trg_domain].to(device)
        
        # Domain indices
        src_idx = 0
        trg_idx = 1
        
        # Random latent code
        z_trg = torch.randn(x_real_src.size(0), latent_dim).to(device)
        
        # Get style codes
        s_trg = mapping_network(z_trg, trg_idx)
        s_src = style_encoder(x_real_src, src_idx)
        
        # Train Discriminator
        optimizer_D.zero_grad()
        
        # Generate fake images
        x_fake = generator(x_real_src, s_trg)
        
        # Real loss
        d_real = discriminator(x_real_trg, trg_idx)
        d_real_loss = criterion_adv(d_real, torch.ones_like(d_real))
        
        # Fake loss
        d_fake = discriminator(x_fake.detach(), trg_idx)
        d_fake_loss = criterion_adv(d_fake, torch.zeros_like(d_fake))
        
        # Total discriminator loss
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        optimizer_D.step()
        
        # Train Generator, Mapping Network, Style Encoder
        optimizer_G.zero_grad()
        
        # Adversarial loss
        d_fake = discriminator(x_fake, trg_idx)
        g_adv_loss = criterion_adv(d_fake, torch.ones_like(d_fake))
        
        # Style reconstruction loss
        s_pred = style_encoder(x_fake, trg_idx)
        g_sty_loss = criterion_style(s_pred, s_trg) * lambda_sty
        
        # Diversity sensitive loss
        z_trg2 = torch.randn(x_real_src.size(0), latent_dim).to(device)
        s_trg2 = mapping_network(z_trg2, trg_idx)
        x_fake2 = generator(x_real_src, s_trg2)
        x_fake2_detach = x_fake2.detach()
        g_ds_loss = -criterion_recon(x_fake, x_fake2_detach) * lambda_ds
        
        # Cycle-consistency loss
        s_src_pred = style_encoder(x_real_src, src_idx)
        x_rec = generator(x_fake, s_src_pred)
        g_cyc_loss = criterion_recon(x_rec, x_real_src) * lambda_cyc
        
        # Total generator loss
        g_loss = g_adv_loss + g_sty_loss + g_ds_loss + g_cyc_loss
        g_loss.backward()
        optimizer_G.step()
        
        # Print losses
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}/{len(dataloader)} | "
                  f"D_loss: {d_loss.item():.4f} | "
                  f"G_loss: {g_loss.item():.4f} | "
                  f"G_adv: {g_adv_loss.item():.4f} | "
                  f"G_sty: {g_sty_loss.item():.4f} | "
                  f"G_ds: {g_ds_loss.item():.4f} | "
                  f"G_cyc: {g_cyc_loss.item():.4f}")
    
    # Evaluation and visualization
    generator.eval()
    with torch.no_grad():
        # Get sample images
        sample_images, _ = fixed_batch
        x_src = sample_images[src_domain].to(device)
        
        # Generate with random style
        z_trg = torch.randn(x_src.size(0), latent_dim).to(device)
        s_trg = mapping_network(z_trg, trg_idx)
        x_fake = generator(x_src, s_trg)
        
        # Denormalize
        x_src = (x_src * 0.5) + 0.5
        x_fake = (x_fake * 0.5) + 0.5
        
        # Create grid
        grid = torch.cat([
            make_grid(x_src, nrow=4, padding=2, normalize=False),
            make_grid(x_fake, nrow=4, padding=2, normalize=False)
        ], dim=1)
        
        # Save images
        save_image(
            grid,
            f"training_progress/starganv2_epoch_{epoch+1:03d}.png"
        )
        
    # Save models periodically
    if (epoch+1) % 10 == 0:
        torch.save({
            'generator': generator.state_dict(),
            'mapping_network': mapping_network.state_dict(),
            'style_encoder': style_encoder.state_dict(),
            'discriminator': discriminator.state_dict()
        }, f"training_progress/starganv2_checkpoint_{epoch+1}.pt")


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mod

Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth


Epoch 1/100:   0%|          | 1/850 [00:01<18:30,  1.31s/it]

Batch 0/850 | D_loss: 1.0246 | G_loss: 1.4342 | G_adv: 0.9146 | G_sty: 0.0334 | G_ds: -0.0047 | G_cyc: 0.4909


...

Epoch 38/100:  21%|██▏       | 181/850 [02:05<09:12,  1.21it/s]

Batch 180/850 | D_loss: 0.2993 | G_loss: -0.0624 | G_adv: 0.7102 | G_sty: 0.0096 | G_ds: -0.8443 | G_cyc: 0.0621


Epoch 38/100:  22%|██▏       | 191/850 [02:12<09:03,  1.21it/s]

Batch 190/850 | D_loss: 0.2609 | G_loss: 0.0430 | G_adv: 0.6243 | G_sty: 0.0106 | G_ds: -0.6619 | G_cyc: 0.0700


Epoch 38/100:  23%|██▎       | 197/850 [02:16<07:26,  1.46it/s]

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arnaud58/selfie2anime")

print("Path to dataset files:", path)

In [ ]:
pip install lpips

In [ ]:
pip install torchmetrics[image] torch-fidelity